In [1]:
import numpy as np
import pandas as pd

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re
import itertools

import string
import re, nltk
from collections import Counter

In [2]:
df = pd.read_csv('../data/MIMIC-III-Final.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52722 entries, 0 to 52721
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  52722 non-null  int64  
 1   TEXT        52722 non-null  object 
 2   HADM_ID     52722 non-null  float64
 3   ICD9_CODE   52722 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


In [4]:
len(df['HADM_ID'].unique())

52722

In [5]:
for p in string.punctuation:
    df['ICD9_CODE'] = df['ICD9_CODE'].apply(lambda x: x.replace(p, ''))

In [6]:
df['DIAG_CODES'] = df['ICD9_CODE'].apply(lambda x: x.replace(' ', ','))

In [7]:
df.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,DIAG_CODES
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,25013 3371 5849 5780 V5867 25063 5363 4580 250...,"25013,3371,5849,5780,V5867,25063,5363,4580,250..."
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,53100 2851 07054 5715 45621 53789 4019 53550 7823,"53100,2851,07054,5715,45621,53789,4019,53550,7823"
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,49320 51881 486 20300 2761 7850 3090 V1251 V1582,"49320,51881,486,20300,2761,7850,3090,V1251,V1582"
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,56081 5570 9973 486 4019,"56081,5570,9973,486,4019"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,41401 99604 4142 25000 27800 V8535 4148 4111 V...,"41401,99604,4142,25000,27800,V8535,4148,4111,V..."


In [8]:
def high_level_code(lst):
    return [code[:3] for code in lst.split(',')]

In [9]:
df['HIGH_LVL_DIAG'] = df['DIAG_CODES'].apply(high_level_code)

In [10]:
df['HIGH_LVL_DIAG'].unique

<bound method Series.unique of 0        [250, 337, 584, 578, V58, 250, 536, 458, 250, ...
1            [531, 285, 070, 571, 456, 537, 401, 535, 782]
2            [493, 518, 486, 203, 276, 785, 309, V12, V15]
3                                [560, 557, 997, 486, 401]
4        [414, 996, 414, 250, 278, V85, 414, 411, V45, ...
                               ...                        
52717        [410, 428, 427, 427, 518, 424, 276, 511, 518]
52718        [486, 428, 518, 397, 496, 416, 585, 427, 276]
52719    [421, 746, 429, 304, 441, 442, V12, 041, 305, ...
52720    [414, 997, 997, 427, 788, 411, V45, E87, 429, ...
52721    [482, 518, 511, 584, 276, 786, 799, 414, V45, ...
Name: HIGH_LVL_DIAG, Length: 52722, dtype: object>

In [11]:
diag_codes = {}
for i, s in enumerate(df['HIGH_LVL_DIAG']):
    for w in s:
        diag_codes[w] = diag_codes.get(w, 1) + 1
diag_codes = {k: v for k, v in sorted(diag_codes.items(), key=lambda item: item[1], reverse=True)}

In [12]:
print('Diag High Level: %d' %(len(diag_codes)))

Diag High Level: 941


In [13]:
icd_groups = {
    0: '001-139',
    1: '140-239',
    2: '240-279',
    3: '280-289',
    4: '290-319',
    5: '320-389',
    6: '390-459',
    7: '460-519',
    8: '520-579',
    9: '580-629',
    10:'630-679',
    11:'680-709',
    12:'710-739',
    13:'740-759',
    14:'760-779',
    15:'780-799',
    16:'800-999',
    17:'V01-V91',
    18:'E000-E999'
}

In [14]:
def convert_hi_lvl(org_lst):
    hi_lvl = []
    for c in org_lst:
        if c.startswith('E'):
            hi_lvl.append(18)
        elif c.startswith('V'):
            hi_lvl.append(17)
        else:
            c = int(c)
            if c >= 800 and c <= 999:
                hi_lvl.append(16)
            elif c >= 780 and c <= 799:
                hi_lvl.append(15)
            elif c >= 760 and c <= 779:
                hi_lvl.append(14)
            elif c >= 740 and c <= 759:
                hi_lvl.append(13)
            elif c >= 710 and c <= 739:
                hi_lvl.append(12)
            elif c >= 680 and c <= 709:
                hi_lvl.append(11)
            elif c >= 630 and c <= 679:
                hi_lvl.append(10)
            elif c >= 580 and c <= 629:
                hi_lvl.append(9)
            elif c >= 520 and c <= 579:
                hi_lvl.append(8)
            elif c >= 460 and c <= 519:
                hi_lvl.append(7)
            elif c >= 390 and c <= 459:
                hi_lvl.append(6)
            elif c >= 320 and c <= 389:
                hi_lvl.append(5)
            elif c >= 290 and c <= 319:
                hi_lvl.append(4)
            elif c >= 280 and c <= 289:
                hi_lvl.append(3)
            elif c >= 240 and c <= 279:
                hi_lvl.append(2)
            elif c >= 140 and c <= 239:
                hi_lvl.append(1)
            elif c >= 1 and c <= 139:
                hi_lvl.append(0)
    return hi_lvl

In [15]:
df['CODED_HIGH_LVL_DIAG'] = df['HIGH_LVL_DIAG'].apply(convert_hi_lvl)

In [16]:
df.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,DIAG_CODES,HIGH_LVL_DIAG,CODED_HIGH_LVL_DIAG
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,25013 3371 5849 5780 V5867 25063 5363 4580 250...,"25013,3371,5849,5780,V5867,25063,5363,4580,250...","[250, 337, 584, 578, V58, 250, 536, 458, 250, ...","[2, 5, 9, 8, 17, 2, 8, 6, 2, 6, 9, 2, 5, 2, 11..."
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,53100 2851 07054 5715 45621 53789 4019 53550 7823,"53100,2851,07054,5715,45621,53789,4019,53550,7823","[531, 285, 070, 571, 456, 537, 401, 535, 782]","[8, 3, 0, 8, 6, 8, 6, 8, 15]"
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,49320 51881 486 20300 2761 7850 3090 V1251 V1582,"49320,51881,486,20300,2761,7850,3090,V1251,V1582","[493, 518, 486, 203, 276, 785, 309, V12, V15]","[7, 7, 7, 1, 2, 15, 4, 17, 17]"
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,56081 5570 9973 486 4019,"56081,5570,9973,486,4019","[560, 557, 997, 486, 401]","[8, 8, 16, 7, 6]"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,41401 99604 4142 25000 27800 V8535 4148 4111 V...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","[414, 996, 414, 250, 278, V85, 414, 411, V45, ...","[6, 16, 6, 2, 2, 17, 6, 6, 17, 17, 3, 6, 2, 6,..."


### Codes to dict

In [17]:
diag_keep = df['ICD9_CODE'].apply(lambda x: x.split(' ')).tolist()

In [18]:
list_sum = sum(diag_keep, [])

In [19]:
count = dict((i, list_sum.count(i)) for i in list_sum)

In [20]:
count

{'25013': 401,
 '3371': 136,
 '5849': 8907,
 '5780': 416,
 'V5867': 2497,
 '25063': 259,
 '5363': 499,
 '4580': 285,
 '25043': 181,
 '40390': 3350,
 '5853': 723,
 '25053': 162,
 '36201': 849,
 '25083': 57,
 '7078': 141,
 'V1351': 45,
 '53100': 59,
 '2851': 4499,
 '07054': 1187,
 '5715': 1250,
 '45621': 409,
 '53789': 331,
 '4019': 20053,
 '53550': 430,
 '7823': 271,
 '49320': 732,
 '51881': 7249,
 '486': 4733,
 '20300': 178,
 '2761': 2985,
 '7850': 588,
 '3090': 48,
 'V1251': 1568,
 'V1582': 2741,
 '56081': 226,
 '5570': 475,
 '9973': 388,
 '41401': 12179,
 '99604': 54,
 '4142': 290,
 '25000': 8784,
 '27800': 1483,
 'V8535': 14,
 '4148': 848,
 '4111': 1649,
 'V4582': 2651,
 '2859': 5296,
 '2720': 5766,
 '4262': 12,
 '44021': 250,
 'V153': 675,
 'V4502': 711,
 '1890': 217,
 '1961': 244,
 '1987': 129,
 '1976': 305,
 '27652': 1347,
 '85206': 85,
 '82111': 23,
 '86403': 28,
 '48242': 158,
 '8600': 355,
 '86121': 394,
 '80703': 123,
 '8220': 34,
 '8052': 298,
 '30560': 381,
 '8900': 20,
 '8

In [21]:
sorted_count = {}
sorted_keys = sorted(count, key=count.get, reverse=True)[:300]

for w in sorted_keys:
    sorted_count[w] = count[w]

In [22]:
sorted_count

{'4019': 20053,
 '4280': 12842,
 '42731': 12594,
 '41401': 12179,
 '5849': 8907,
 '25000': 8784,
 '2724': 8504,
 '51881': 7249,
 '5990': 6442,
 '53081': 6156,
 '2720': 5766,
 '2859': 5296,
 '2449': 4788,
 '486': 4733,
 '2851': 4499,
 '2762': 4358,
 '496': 4296,
 '99592': 3792,
 'V5861': 3698,
 '5070': 3592,
 '0389': 3580,
 '5859': 3367,
 '40390': 3350,
 '311': 3347,
 '3051': 3272,
 '412': 3203,
 '2875': 3002,
 '41071': 3001,
 '2761': 2985,
 'V4581': 2943,
 '4240': 2878,
 'V1582': 2741,
 '5119': 2693,
 'V4582': 2651,
 '40391': 2566,
 'V290': 2529,
 '4241': 2517,
 '78552': 2501,
 'V5867': 2497,
 '42789': 2396,
 '32723': 2328,
 '9971': 2313,
 '5845': 2223,
 '2760': 2221,
 '7742': 2183,
 '5180': 2137,
 'V053': 2119,
 '4168': 2118,
 '49390': 2113,
 '2767': 2113,
 '45829': 2094,
 '2749': 2033,
 '4589': 1992,
 '73300': 1910,
 '78039': 1879,
 '5856': 1868,
 '5185': 1778,
 '4271': 1767,
 '4254': 1666,
 '4111': 1649,
 'V1251': 1568,
 '30000': 1552,
 '3572': 1552,
 '99811': 1508,
 '27800': 1483,


In [23]:
diag2idx = {}
for d in sorted_count.keys():
    if d not in diag2idx:
        diag2idx[d] = len(diag2idx)

### Convert code list

In [24]:
def diag_code2idx(org_lst):
    coded_lst = []
    for c in org_lst.split(','):
        if c in diag2idx:
            coded_lst.append(diag2idx[c])
    return coded_lst

In [25]:
df['CODED_DIAG'] = df['DIAG_CODES'].apply(diag_code2idx)

In [26]:
df.head()

,Unnamed: 0,TEXT,HADM_ID,ICD9_CODE,DIAG_CODES,HIGH_LVL_DIAG,CODED_HIGH_LVL_DIAG,CODED_DIAG
0,0,{Admission Date: [**2117-9-11**] ...,100001.0,25013 3371 5849 5780 V5867 25063 5363 4580 250...,"25013,3371,5849,5780,V5867,25063,5363,4580,250...","[250, 337, 584, 578, V58, 250, 536, 458, 250, ...","[2, 5, 9, 8, 17, 2, 8, 6, 2, 6, 9, 2, 5, 2, 11...","[299, 4, 291, 38, 238, 22, 165, 140]"
1,1,{Admission Date: [**2150-4-17**] ...,100003.0,53100 2851 07054 5715 45621 53789 4019 53550 7823,"53100,2851,07054,5715,45621,53789,4019,53550,7823","[531, 285, 070, 571, 456, 537, 401, 535, 782]","[8, 3, 0, 8, 6, 8, 6, 8, 15]","[14, 96, 88, 296, 0, 272]"
2,2,{Admission Date: [**2108-4-6**] Dischar...,100006.0,49320 51881 486 20300 2761 7850 3090 V1251 V1582,"49320,51881,486,20300,2761,7850,3090,V1251,V1582","[493, 518, 486, 203, 276, 785, 309, V12, V15]","[7, 7, 7, 1, 2, 15, 4, 17, 17]","[161, 7, 13, 28, 202, 60, 31]"
3,3,{Admission Date: [**2145-3-31**] ...,100007.0,56081 5570 9973 486 4019,"56081,5570,9973,486,4019","[560, 557, 997, 486, 401]","[8, 8, 16, 7, 6]","[250, 13, 0]"
4,4,{Admission Date: [**2162-5-16**] ...,100009.0,41401 99604 4142 25000 27800 V8535 4148 4111 V...,"41401,99604,4142,25000,27800,V8535,4148,4111,V...","[414, 996, 414, 250, 278, V85, 414, 411, V45, ...","[6, 16, 6, 2, 2, 17, 6, 6, 17, 17, 3, 6, 2, 6,...","[3, 5, 64, 141, 59, 33, 31, 11, 0, 10, 177, 38..."


### Notes preprocessing

In [27]:
def clean_text(text):
    return [x for x in list(itertools.chain.from_iterable([t.split("<>") for t in text.replace("\n"," ").split("|")])) if len(x) > 0]

In [28]:
most_frequent_tags = [re.match("^(.*?):",x).group() for text in df.TEXT for x in text.split("\n\n") if pd.notnull(re.match("^(.*?):",x))]

In [29]:
pd.Series(most_frequent_tags).value_counts().head(10)

{Admission Date:                         51667
Date of Birth:                           50930
Service:                                 50575
Allergies:                               41157
Attending:                               41016
Discharge Diagnosis:                     38639
Major Surgical or Invasive Procedure:    34917
Physical Exam:                           29175
Followup Instructions:                   28967
Facility:                                26668
dtype: int64

In [30]:
irrelevant_tags = ['Admission Date:', 'Date of Birth:', 'Service:', 'Allergies:', 'Attending:',
                  'Discharge Diagnosis:', 'Major Surgical or Invasive Procedure:', 'Physical Exam:',
                  'Followup Instructions:', 'Facility:']

updated_text = ["<>".join(["|".join(re.split("\n\d|\n\s+",re.sub("^(.*?):","",x).strip())) for x in text.split("\n\n") if pd.notnull(re.match("^(.*?):",x)) and re.match("^(.*?):",x).group() not in irrelevant_tags ]) for text in df.TEXT]
updated_text = [re.sub("(\[.*?\])", "", text) for text in updated_text]

updated_text = ["|".join(clean_text(x)) for x in updated_text]

In [31]:
df["CLEAN_WORDS"] = updated_text

In [32]:
df["CLEAN_WORDS"] = [" ".join([y for y in x.split("|") if len(y.split()) > 3]) for x in df["CLEAN_WORDS"]]

In [33]:
def remove_stopwords(text): 
        stop_words = set(stopwords.words("english")) 
        word_tokens = word_tokenize(text) 
        filtered_text = [word for word in word_tokens if word not in stop_words] 
        return filtered_text 

In [34]:
def preprocess(note):
    note = note.replace('\n',' ')
    note = note.replace('w/', 'with')
    note = note.lower() #lower case
    note = re.sub(r'\d+', '', note) #remove numbers
    note = note.translate(str.maketrans('', '', string.punctuation)) #remove punctuation
    note = " ".join(note.split())
    note = remove_stopwords(note)
    return note

In [35]:
df['CLEAN_WORDS'] = df['CLEAN_WORDS'].apply(preprocess)

In [36]:
df.to_csv('../data/full_data.csv')

In [37]:
sample_20k = df.sample(n=20000)
sample_20k.to_csv('../data/clean.csv')

In [38]:
sample_2k = df.sample(n=2000)
sample_2k.to_csv('../data/clean_2k.csv')